# Cholesky Decomposition for Financial Simulations
This notebook demonstrates how to use Cholesky decomposition to simulate correlated financial returns. This is particularly useful for backtesting trading strategies and risk management.
**Note that: Please download and run notebook for graphs at the end of the notebook.**

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import string
from ipywidgets import FloatSlider, IntSlider, VBox, Layout
from bqplot import LinearScale, Lines, Axis, Figure

Widget Display setup

In [2]:
%%html

<style> .widget-readout{ color:black; } </style>

## Introduction
When backtesting trading strategies, it's crucial to test them against simulated data that maintains realistic correlations between assets. This helps avoid overfitting and provides more robust strategy validation. The Cholesky decomposition is a mathematical technique that allows us to generate correlated random variables while maintaining specific correlation structures.

### Creating basic matrix

Here we create a 3x3 correlation matrix with correlation of 0.7 between assets:

In [3]:
corr = np.array([[1, 0.7, 0.7], [0.7, 1, 0.7], [0.7, 0.7, 1]])

corr

array([[1. , 0.7, 0.7],
       [0.7, 1. , 0.7],
       [0.7, 0.7, 1. ]])

### Perform Cholesky Decomposition
The Cholesky decomposition finds a lower triangular matrix L such that L × L.T = correlation matrix

In [4]:
chol = np.linalg.cholesky(corr)
chol

array([[1.        , 0.        , 0.        ],
       [0.7       , 0.71414284, 0.        ],
       [0.7       , 0.29405882, 0.65079137]])

corr = chol * chol.T

In [5]:
np.matmul(chol, chol.T)

array([[1. , 0.7, 0.7],
       [0.7, 1. , 0.7],
       [0.7, 0.7, 1. ]])

### Creating Normal Distrubitued Data 

In [6]:
rand_data = np.random.normal(size=(3,1000))
rand_data

array([[ 0.54233524, -0.38884988,  0.72221971, ..., -0.25539414,
        -1.75705012,  1.07834648],
       [-1.0553344 , -0.96697366,  0.28922404, ...,  1.96175704,
        -0.36244766, -0.47937933],
       [ 0.05893948, -0.93617148,  0.18235105, ...,  0.2333712 ,
         1.21458495, -1.25677681]])

In [7]:
pd.DataFrame(rand_data.T).corr()

,0,1,2
0,1.000000,-0.013886,-0.019168
1,-0.013886,1.000000,0.026578
2,-0.019168,0.026578,1.000000


In [8]:
no_corr_data = pd.DataFrame(rand_data, index = ['A', 'B', 'C']).T/100

### Visualizing Uncorrelated Results 

In [9]:
sim_cum_rets_plot_no_corr = px.line((1+no_corr_data).cumprod(), title='Simulated returns with no correlation', width=1000, height=500)
sim_cum_rets_plot_no_corr.show()

In [10]:
sim_corr_rets = pd.DataFrame(np.matmul(chol, rand_data), index = ['A', 'B', 'C']).T/100

In [11]:
sim_corr_rets.head()

,A,B,C
0,0.005423,-0.003740,0.001077
1,-0.003888,-0.009628,-0.011658
2,0.007222,0.007121,0.007093
3,-0.004272,-0.001027,0.002036
4,0.001766,-0.003088,-0.005750


### Visualizing Correlated Results

In [12]:
sim_cum_rets_plot = px.line((1+sim_corr_rets).cumprod(), title='Simulated returns with correlation', width=1000, height=500)
sim_cum_rets_plot.show()

In [13]:
sim_corr_rets.corr()

,A,B,C
A,1.000000,0.686161,0.685552
B,0.686161,1.000000,0.702716
C,0.685552,0.702716,1.000000


### Creating Slider for Widget Tool


Creating interactive sliders to allow interactive manipulation of:

* Correlation between assets
* Number of securities
* Sample size

In [14]:
corr_slider = FloatSlider(min =0, max =0.99, value= 0.7, step=0.01, description = 'Correlation', continuous_update=False, layout = {'fontcolor':'red'})
num_secs_slider = IntSlider(min = 2, max = 10, value = 3, description = '# Securities', continuous_update=False)
sample_size_slider = IntSlider(min = 50, max = 3000, value = 1000, description = 'Sample Size', continuous_update=False)

corr_slider.style.handle_color = 'orange'
num_secs_slider.style.handle_color = 'blue'
sample_size_slider.style.handle_color = 'red'

### Initialize Simulation
Set up initial simulation parameters and generate first set of returns

In [15]:
corr = np.full((num_secs_slider.value,num_secs_slider.value),corr_slider.value)
np.fill_diagonal(corr,1)
chol = np.linalg.cholesky(corr)
rand_data = np.random.normal(size=(num_secs_slider.value,sample_size_slider.value))
sim_corr_rets = pd.DataFrame(np.matmul(chol, rand_data), index = list(string.ascii_uppercase)[:num_secs_slider.value]).T/100
cum_prod_rets = (1+sim_corr_rets).cumprod()

### Update Function
Define function to update the simulation when parameters change:

In [16]:
def update_matrix(caller):
    corr = np.full((num_secs_slider.value,num_secs_slider.value),corr_slider.value)
    np.fill_diagonal(corr,1)
    chol = np.linalg.cholesky(corr)
    rand_data = np.random.normal(size=(num_secs_slider.value,sample_size_slider.value))
    sim_corr_rets = pd.DataFrame(np.matmul(chol, rand_data), index = list(string.ascii_uppercase)[:num_secs_slider.value]).T/100
    cum_prod_rets = (1+sim_corr_rets).cumprod()
    line.x = cum_prod_rets.index
    line.y = cum_prod_rets.values.T

num_secs_slider.observe(update_matrix, 'value')
corr_slider.observe(update_matrix, 'value')
sample_size_slider.observe(update_matrix, 'value')

### Interactive Visualization Setup
Create an interactive plot that updates with slider changes:

In [17]:
%matplotlib widget
sc_x = LinearScale()
sc_y = LinearScale()
line = Lines(x=cum_prod_rets.index, y=cum_prod_rets.values.T,
             scales={'x': sc_x, 'y': sc_y})
ax_x = Axis(scale=sc_x, label='Index',label_color = 'white')
ax_y = Axis(scale=sc_y, orientation='vertical', label='Cumulative Returns', label_color = 'white')
fig = Figure(marks=[line], axes=[ax_x, ax_y], title='Correlated Returns Simulator', title_style = {'fill': 'white'}, animation_duration=500)

### Simulation Visualization

In [18]:
VBox([num_secs_slider, corr_slider, sample_size_slider, fig])